# Parsing Julia to Tapenade's IL

This notebook shows how to parse julia code into the format used by tapenade in a way to start developping source transform adjoint code. 

## Load Tapenade's tree structure
Start with compiling

In [ ]:
#= 
Add some info and some calls to the makefile implemented in this example
=#

Load the newly created library

In [ ]:
using Libdl
c_libname = "./julialib.so"
lib = Libdl.dlopen(c_libname) # Open the library explicitly.

Define function handler to useful functions.

In [ ]:
# Functions from the tree builder library for parsing the julia tree
c_newJuliaTreeBuilder = Libdl.dlsym(lib, :newJuliaTreeBuilder)   # Get a symbol for the function to call. !!!! Redefine this library
c_startJuliaTree = Libdl.dlsym(lib, :startJuliaTree)
c_turnListFrontier = Libdl.dlsym(lib, :turnListFrontier)
## To review: c_startAnnotation = Libdl.dlsym(lib, :startTree) ### This is an extern function, and we might want to have this pointing to another lib!
c_putValue = Libdl.dlsym(lib, :putValue) ## Also extern
c_putTree = Libdl.dlsym(lib, :putTree) ## Also extern
c_oneLessWaiting = Libdl.dlsym(lib, :oneLessWaiting) 
c_putListTree = Libdl.dlsym(lib, :putListTree) ## Extern
c_startDummyTree = Libdl.dlsym(lib, :startDummyTree) ## Extern
c_terminateListTree = Libdl.dlsym(lib, :terminateListTree) ## Extern
c_getTreeBuilt = Libdl.dlsym(lib, :getTreeBuilt) ## Extern
c_getListTreeBuilt = Libdl.dlsym(lib, :getListTreeBuilt) ## Extern
c_deleteTreeBuilder = Libdl.dlsym(lib, :deleteTreeBuilder) ## Extern
c_resetTreeBuilder = Libdl.dlsym(lib, :resetTreeBuilder) ## Extern
c_removeTree = Libdl.dlsym(lib, :removeTree)
c_showTreeBuilderState = Libdl.dlsym(lib, :showTreeBuilderState) ## Debug purposes

# Functions from cvtp to help producing the tree

## Create the associated structures on Julia's side

In [ ]:
juliaTreeBuilder = ccall(c_newJuliaTreeBuilder, Ptr{Cvoid}, (Cint,), 5)

In [ ]:
ccall(c_startJuliaTree, Ptr{Cvoid}, (Ptr{Cvoid}, Cstring,), juliaTreeBuilder, "assign")

In [ ]:
ccall(c_showTreeBuilderState, Cvoid, (Ptr{Cvoid},), juliaTreeBuilder)

In [ ]:
ccall(c_startJuliaTree, Ptr{Cvoid}, (Ptr{Cvoid}, Cstring,), juliaTreeBuilder, "ident")

In [ ]:
ccall(c_showTreeBuilderState, Cvoid, (Ptr{Cvoid},), juliaTreeBuilder)

In [ ]:
ccall(c_putValue, Cvoid, (Ptr{Cvoid}, Cstring), juliaTreeBuilder, "toto")

In [ ]:
ccall(c_startJuliaTree, Ptr{Cvoid}, (Ptr{Cvoid}, Cstring,), juliaTreeBuilder, "int32")

In [ ]:
ccall(c_putValue, Cvoid, (Ptr{Cvoid}, Cstring), juliaTreeBuilder, "42")

In [ ]:
ccall(c_showTreeBuilderState, Cvoid, (Ptr{Cvoid},), juliaTreeBuilder)

In [ ]:
using DataStructures

In [ ]:
ct_testfile = "ct_test.jl"
orig_testfile = "kepler.jl"
basic_test = "testfile.jl"

In [ ]:
EndOfList = :EndOfList
cstSymb = :cst

In [ ]:
metalnames = Dict(
    :(=) => "assign",
    :. => "package", # Check whether the dot refers to a local path or is synonym for an import
    :block => "block",
    :vect => "vect",
    :vcat => "vcat",
    :return => "return",
    :tuple => "tuple",
    :ref => "ref",
    :curly => "curly",
    :call => "call", 
    :function => "function",
    :using => "using", 
    :toplevel => "toplevel",
    Int64 => "int64",
    Float64 => "float64",
    Int32 => "int32",
    Float32 => "float32",
    LineNumberNode => "linenumber",
    Nothing => "none",
    :EndOfList => "EndOfList"
)

In [ ]:
typeof(EndOfList)

In [ ]:
function file2ast(fname::AbstractString, outfile::AbstractString)
    
    # tree_builder = ccall(c_newJuliaTreeBuilder, Ptr{Cvoid}, (Cint,), 5)
    all_symbols = Set()
    open(outfile, "w") do out 
        code = read(fname, String)
        ## println(code)

        exp_in_waiting = Stack{Any}()
        
        prev = 0
        next = 1
        while next > prev 
            prev = next
            exp, next = Meta.parse(code, prev)
            push!(exp_in_waiting, exp)
            ## println("     ***** Currently looking at: ")
            ## println(exp)
            while !isempty(exp_in_waiting)
                e = pop!(exp_in_waiting)
                if e isa Expr 
                    # Parse expression
                    # Write current instruction
                    println("     ***** Currently looking at: ", string(e))
                    # println(e.head)
                    println("Operator ", e.head, " has ", length(e.args), " children. ")
                    # println(e.args)
                    # If Current instruction is a call, make sure to add an end of list
                    # Same thing holds for a 'using' statement
                    #####################
                    # NOTE:             #
                    # One might need    #
                    # to check if other #
                    # keywords need this#
                    #####################
                    #=if e.head == (:.)
                    #if e.head in (:.,)
                        println("We are here! ", e)
                    end =#
                    if e.head in (:call, :tuple, :using, :vect) 
                        push!(exp_in_waiting, EndOfList)
                    end
                    for arg in Iterators.reverse(e.args)
                        push!(exp_in_waiting, arg)
                    end
                    push!(all_symbols, metalnames[e.head]) 
                    push!(exp_in_waiting, e.head) 
                    # write(out, e.head) # New C node from tree builder
                    # write(out, "\n")
                elseif e isa Symbol
                    # Add symbol to file
                    #println("This is the current symbol: ", e)
                    if haskey(metalnames, e) 
                        write(out, metalnames[e]) # New leaf for current operator
                    else
                        write(out, "ident\n")
                        write(out, e) # New leaf for current operator
                    end
                    write(out, "\n")
                else
                    # Neither a symbol nor an expression, it's probably a constant
                    ## println("Type of our constant thing is ", typeof(e))
                    push!(all_symbols, metalnames[typeof(e)])
                    write(out, metalnames[typeof(e)])
                    write(out, "\n")
                    ## println("Looking at cst things")
                    ## println(e)
                    write(out, string(e))
                    write(out, "\n")
                end
            end
                    
            
        end
    end
    return all_symbols
end

In [ ]:
all_symbols = file2ast(basic_test, "outputtest")

In [ ]:
e = Meta.parse("using LinearAlgebra")

In [ ]:
e.head

In [ ]:
e.args

In [ ]:
e.args[1].head

In [ ]:
g = Meta.parse("using LinearAlgebra, ForwardDiff")

In [ ]:
g.args

In [ ]:
f = Meta.parse("cos.(vect)")

In [ ]:
f.head

In [ ]:
f.args

In [ ]:
typeof(:($(Expr(:., :LinearAlgebra))))

In [ ]:
@eval ($(Expr(:., :LinearAlgebra)))

In [ ]:
Expr(:., :LinearAlgebra)